In [ ]:
import sys
sys.path.append('../../..')
sys.path.append('../..')
sys.path.append('..')

from selective_recruitment.plotting import make_scatterplot
from selective_recruitment.scripts.script_mapwise import calc_ttest_mean
import selective_recruitment.data as ss
import selective_recruitment.regress as ra
import selective_recruitment.region as sroi

from statsmodels.stats.anova import AnovaRM  # perform F test
from scipy.stats import ttest_rel

from pathlib import Path
import pandas as pd
import numpy as np



In [ ]:
# setting working directory
wkdir = 'A:\data\Cerebellum\CerebellumWorkingMemory\selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/srv/diedrichsen/data/Cerebellum/CerebellumWorkingMemory/selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/Users/jdiedrichsen/Data/wm_cerebellum/selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/Volumes/diedrichsen_data$/data/Cerebellum/CerebellumWorkingMemory/selective_recruit'

# selective recruitment: ROI-based
scatterplots and statistical tests on the residuals for the selected region of interest

In [ ]:
# setting up names and colors
label_dict = {1: 'LfLs', 2: 'LfMs',
              3: 'LfHs', 4: 'MfLs',
              5: 'HfLs', 6:'rest'}
marker_dict = {1: '^', 2: 'X',
               3: 'X', 4: 'o',
               5: 'o', 6: 's'}
color_dict = {1: 'grey', 2: 'r',
              3: 'r', 4: 'b',
              5: 'b', 6: 'g'}

## Prepare the data for the scatterplot and the test

* Pick one parcellation: Here NettekovenSym32, a data-driven hierarchical symmetric parcellation. ROI will be selected from this parcellation.
* ss.get_summary_conn first uses a connectivity model to get predicted values for all the cerebellar voxels and then summarize the predicted values by averaging the values within the selected region of interest.


In [ ]:
# get the summary df with observed and predicted activation within cerebellar parcels
dataset = "WMFS"
ses_id = 'ses-01'
parcellation = "NettekovenSym32"
cortex = "Icosahedron1002"
D = ss.get_summary_conn(dataset = dataset,
                     ses_id = ses_id,
                     subj = None, # to do all the subjects
                     atlas_space = "SUIT3",
                     cerebellum_roi = parcellation,
                     cerebellum_roi_selected = None,
                     cortex_roi = "Icosahedron1002",
                     type = "CondHalf",
                     add_rest = True,
                     mname_base = 'MDTB_all_Icosahedron_L2regression',
                     mname_ext = '_A8',
                     crossed = True)
D = ra.roi_regress(D, fit_intercept=True)
D.to_csv(wkdir + f'/fs_ROI_{parcellation}_conn_reg.tsv', sep='\t')
D = pd.read_csv(wkdir + f'/fs_ROI_{parcellation}_conn_reg.tsv', sep='\t')

### Select region of interest.
Here we pick one parcel by first focusing on one network, the most relevant network: here the motor network. From the sub-regions within that network, we pick the one that shows consistently high activation throughout the task

In [ ]:
roi_exp = "M.." # roi_exp = "D.?3.?R|D.?3.?L" # roi_exp = "D.?3.?R"
_, _, selected_ = sroi.get_parcel_single(parcellation = parcellation, 
                                               atlas_space = "SUIT3",
                                               roi_exp = roi_exp)
print(selected_)
table = pd.pivot_table(D.loc[np.isin(D.roi_name.values, selected_)], values='Y', index=['roi_name'], aggfunc=np.mean)
sorted_regions = table.sort_values(by=["Y"],ascending=False)
display(sorted_regions.T)
roi_selected = sorted_regions.T.columns[0]
print(f"selected region is {roi_selected}")


### make the scatterplot and test the residuals

In [ ]:
# get the dataframe with the selected region
df = D.loc[D.roi_name == "M3R"]
make_scatterplot(df, split='cond_num',
                labels=label_dict,
                markers=marker_dict,
                colors=color_dict)
# # plt.xlabel("predicted cerebellar activation")
# # # plt.savefig("working_memory_D3R_scatter.pdf")
print(AnovaRM(data=df[df.cond_name != 'rest'], depvar='res',subject='sn', within=['cond_name'], aggregate_func=np.mean).fit())